## データローダー

In [272]:
#REQUIREMENT
import pickle
import matplotlib.pyplot as plt
import re
import time
import random
from gensim.models.word2vec import Word2Vec
import matplotlib.pyplot as plt
import matplotlib as mpl
font = {"family":'ipag'}
mpl.rc('font', **font)
import MeCab
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence,pad_packed_sequence
from nltk import bleu_score


In [273]:
class GenDataLoader(object):
    def __init__(self, file_path, batch_size):
        super(GenDataLoader, self).__init__()
        self.data = self.load_file(file_path)
        self.batch_size = batch_size
        self.pointer = 0
        self.data_num = len(self.data)
        
        self.reset()
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.pointer >= self.data_num:
            self.reset()
            raise StopIteration
        batch = torch.tensor(self.data[self.pointer : self.pointer+self.batch_size], 
                             dtype=torch.long)
        batch_X = batch[:, :-1].to(device)
        batch_Y = batch[:, 1:].to(device)
        self.pointer += self.batch_size
        return batch_X, batch_Y
    
    def load_file(self, file_path):
        data = []
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip().split()
                line = [int(x) for x in line]
                data.append(line)
        return data
    
    def reset(self):
        self.pointer = 0
        random.shuffle(self.data)
        
class DisDataLoader(object):
    def __init__(self, positive_file, negative_file, batch_size):
        super(DisDataLoader, self).__init__()
        self.batch_size = batch_size
        pos_data = self.load_file(positive_file)
        neg_data = self.load_file(negative_file)
        self.data = pos_data + neg_data
        
        self.labels = [1 for _ in range(len(pos_data))] + [0 for _ in range(len(neg_data))]
        self.pairs = list(zip(self.data, self.labels))
        self.data_num = len(self.pairs)
        self.pointer = 0
        self.reset()
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.pointer >= self.data_num:
            self.reset()
            raise StopIteration
        
        batch_X, batch_Y = zip(*self.pairs[self.pointer : self.pointer+self.batch_size])
        batch_X = torch.tensor(batch_X, dtype=torch.long, device=device)
        batch_Y = torch.tensor(batch_Y, dtype=torch.long, device=device)
        self.pointer += self.batch_size
        return batch_X, batch_Y
            
    def load_file(self, file_path):
        data = []
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip().split()
                line = [int(x) for x in line]
                data.append(line)
        return data
    
    def reset(self):
        self.pointer = 0
        random.shuffle(self.pairs)

# generator

In [274]:
class Generator(nn.Module):
    def __init__(self, vocab_size, batch_size, 
                 embedding_size, hidden_size, max_length):
        super(Generator, self).__init__()
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.max_length = max_length
        
        self.embedding= nn.Embedding(self.vocab_size, self.embedding_size)
        
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, num_layers=1,
                           batch_first=True)
        self.linear = nn.Linear(self.hidden_size, self.vocab_size)
        
    def forward(self, seqs):
        N = seqs.size(0)
        embed = self.embedding(seqs)
        h, c = self.init_hidden(N)
        self.lstm.flatten_parameters()
        hidden, (h, c) =self.lstm(embed, (h, c))
        lin =self.linear(hidden)
        outputs = F.log_softmax(lin, dim=-1)
        outputs =outputs.view(-1, self.vocab_size)
        return outputs
    
    def step(self, x, h, c):
        embed =self.embedding(x)
        self.lstm.flatten_parameters()
        hidden, (h, c) = self.lstm(embed, (h, c))
        pred =F.softmax(self.linear(hidden), dim=-1)
        return pred, h, c
    
    def sample(self, x=None):
        flag =False
        if x is None:
            flag = True
        if flag:
            x = torch.empty(self.batch_size, 1).fill_(BOS).long().to(device)
        h, c = self.init_hidden(self.batch_size)
        
        samples = []
        if flag:
            for i in range(self.max_length):
                output, h, c = self.step(x, h, c)
                output = output.squeeze(1)
                x = output.multinomial(1)
                samples.append(x)
                
        else:
            given_len = x.size(1)
            lis = x.chunk(x.size(1) ,dim=1)
            for i in range(given_len):
                output, h, c = self.step(lis[i], h, c)
                samples.append(lis[i])
            output = output.squeeze(1)
            x = output.multinomial(1)
            for i in range(given_len, self.max_length):
                samples.append(x)
                output, h, c = self.step(x, h, c)
                output = output.squeeze(1)
                x = output.multinomial(1)
        output = torch.cat(samples, dim=1)
        return output
    
    def init_hidden(self, N):
        h0 = torch.zeros(1, N, self.hidden_size).to(device)
        c0 = torch.zeros(1, N, self.hidden_size).to(device)
        return h0, c0

# Discriminator

In [275]:
class Highway(nn.Module):
    def __init__(self, input_size, num_layers=1, f=F.relu):
        super(Highway, self).__init__()
        self.num_layers = num_layers
        self.linear1 = nn.ModuleList([nn.Linear(input_size, input_size) for _ in range(num_layers)])
        self.linear2 = nn.ModuleList([nn.Linear(input_size, input_size) for _ in range(num_layers)])
        self.f = f
        
    def forward(self, x):
        for layer in range(self.num_layers):
            gate = torch.sigmoid(self.linear1[layer](x))
            nonlinear =self.f(self.linear2[layer](x))
            x = gate * nonlinear + (1 - gate) * x
        return x
    
class Discriminator(nn.Module):
    def __init__(self, vocab_size, batch_size, embedding_size,
                num_filters, filter_sizes, dropout_prob=0.75):
        super(Discriminator, self).__init__()
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        assert len(self.num_filters) ==len(self.filter_sizes)
        
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        self.convs = nn.ModuleList([nn.Conv2d(1, Co, (K, self.embedding_size))
                                  for Co, K in zip(self.num_filters, self.filter_sizes)])
        
        self.highway = Highway(sum(self.num_filters))
        
        self.dropout = nn.Dropout(dropout_prob)
        self.linear = nn.Linear(sum(self.num_filters), 2)
        
    def forward(self, x, log=True):
        x = self.embedding(x)
        x = x.unsqueeze(1)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs]
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        x = torch.cat(x, 1)
        x = self.highway(x)
        x = self.dropout(x)
        x = self.linear(x)
        if log:
            x = F.log_softmax(x, dim=-1)
        else:
            x = F.softmax(x, dim=-1)
        return x

# Monte Carlo search

In [ ]:
class Rollout():
    def __init__(self, model, update_rate):
        self.ori_model = model
        self.own_model = copy.deepcopy(model)
        self.update_rate = update_rate
        
    def get_reward(self, x, num, discriminator):
        rewards = []
        batch_size = x.size(0)
        seq_length = x.size(1)
        for i in range(num):
            for t in range(1, seq_length):
                data = x[:, 0:t]
                samples = self.own_model.sample(x=data)
                pred = discriminator(samples, log=False)
                pred = pred.cpu().data[:, 1].numpy()
                if i == 0:
                    rewards.append(pred)
                else:
                    rewards[t-1] += pred
            pred = discriminator(x, log=False)
            pred = pred.cpu().data[:, 1].numpy()
            if i == 0:
                rewards.append(pred)
            else:
                rewards[seq_length-1] += pred
            
        rewards = np.array(rewards)
        rewarda = np.transpose(rewards) / (1.0*num)
        return rewards
        
    def update_params(self):
        dic = {}
        for name, param in self.own_model.named_parameters():
            dic[name] = param.data
        for name, param in self.ori_model.named_parameters():
            if name.startswith('emb'):
                param.data = dic[name]
            else:
                param.data = (1- self.update_rate) * param.data + self.update_rate * dic[name]
        self.own_model = copy.deepcopy(self.ori_model)    

In [ ]:
#PATH
PROJECT_PATH = "../../../../project/question_generator"
DATA_PATH=PROJECT_PATH + "/data/takken/"
MODEL_PATH="./saved_model/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#DEVICE
if device == torch.device('cuda'):
    print('deviceはGPUを使用します')
else:
    print('deviceはCPUを使用します')


# In[3]:


#データのロード
def load_data(file_path):
    data = []
    for line in open(file_path, encoding='utf-8'):
        words = line.strip().split()
        data.append(words)
    return data


# In[4]:


#Vocabクラス
##wordからidを生成
##idからwordを生成
class Vocab(object):
    def __init__(self, word2id={}):
        """
        word2id: 単語(str)をインデックス(int)に変換する辞書
        id2word: インデックス(int)を単語(str)に変換する辞書
        """
        self.word2id = dict(word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}    
        
    def build_vocab(self, sentences, min_count=10):
        """ 各単語の出現回数の辞書を作成する
        :param senteces(str) 文章
        :param min_count(int) 最低カウント数(これ以下の単語は辞書に含めない)
        """
        word_counter = {}
        for sentence in sentences:
            for word in sentence:
                word_counter[word] = word_counter.get(word, 0) + 1

        # min_count回以上出現する単語のみ語彙に加える
        for word, count in sorted(word_counter.items(), key=lambda x: -x[1]):
            if count < min_count:
                break
            _id = len(self.word2id)
            self.word2id.setdefault(word, _id)
            self.id2word[_id] = word
            
def remove_choice_number(text):
    '''文頭に選択肢番号がついている場合それを除く。
    前処理で使うだけなのでこのファイルでは呼び出さない。別のファイルに移したい。
    '''
    remove_list = [
        "^ア ", "^イ ", "^ウ ", "^エ ", "^オ ", "^1 ", "^2 ", "^3 ", "^4 ", "^5 "
    ]
    for i, word in enumerate(remove_list):
        text = re.sub(word, "", text)
    return text


def remove_symbol(text):
    '''
    入力されたテキストから句読点などの不要な記号をいくつか削除する。
    '''
    remove_list = [
        ',', '.', '-', '、', '，', '。', '\ufeff', '\u3000', '「', '」', '（', '）',
        '(', ')','\n'
    ]
    for i, symbol in enumerate(remove_list):
        text = text.replace(symbol, '')
    return text


def add_bos_eos(text):
    '''
    文章の先頭に<BOS>、<EOS>を加える。文末の改行コードの都合で<EOS>の直前にはスペースを入れていない。
    '''
    return "<BOS> " + text + "<EOS>"


def replace_number(text):
    '''textの数値表現を<Number>トークンに置き換える
    textは分かち書きされていること
    '''
    new_text = ""
    for word in text.split(' '):
        if word.isnumeric():
            new_text += "<NUM> "
        elif word == "<EOS>":
            new_text += "<EOS>"
        else:
            new_text += word + " "
    return new_text


def isalpha(s):
    '''
    Args:
        s:string
    Returns:
        bool:sが半角英字から成るかどうか
    '''
    alphaReg = re.compile(r'^[a-zA-Z]+$')
    return alphaReg.match(s) is not None


def replace_alphabet(text):
    '''
    Args:
    text:分かち書きされた文。
    Return:
    textの数値表現をAに置き換える
    '''
    new_text = ""
    for word in text.split(' '):
        if isalpha(word):
            new_text += "A "
        elif word == "<EOS>":
            new_text += word
        else:
            new_text += word + " "
    return new_text


# In[13]:


#########ここもまとめたい
###
#過去12年分の宅建の過去問
takken = pd.read_csv(DATA_PATH+"takken.csv", encoding='utf-8')
mondaishu = pd.read_csv(DATA_PATH+"mondaishu.csv", encoding='utf-8')
nikken = pd.read_csv(DATA_PATH+"nikken.csv",encoding='utf-8')
legal_mind = pd.read_csv(DATA_PATH+"legal_mind.csv",encoding='utf-8')

#データをまとめる
takken = takken[["Question", "Choice"]]
ocr = pd.concat([mondaishu,nikken,legal_mind],axis=0,ignore_index=True)
ocr = ocr[["Wakati_Question","Wakati_Choice"]]
ocr.columns = ["Question","Choice"]

#データをMeCabで分かち書きして、不要な文字の除去、BOS,EOSの追加を行う
m = MeCab.Tagger("-Owakati")
takken = takken.applymap(remove_symbol)
ocr = ocr.applymap(remove_symbol)
takken = takken.applymap(m.parse)
takken = pd.concat([takken,ocr],axis=0,ignore_index=True)
takken = takken.applymap(remove_symbol)
takken = takken.applymap(add_bos_eos)
takken = takken.applymap(replace_number)
takken = takken.applymap(replace_alphabet)
print("data size is",len(takken))
#takken.head()


################################################################
#改良１
#選択股と質問文で長さ指定してみる(本来これでうまくいくはずなのだが)
#まずは同じ長さ→別の長さにすることを目指す
max_length_question=100
max_length_choice=100

takken = takken[takken["Question"].str.split(' ').apply(len) <= max_length_question]
takken = takken[takken["Choice"].str.split(' ').apply(len) <= max_length_choice]
takken.reset_index(drop=True, inplace=True)

data_question = []
data_choice = []
for i in range(len(takken)):
    data_sentence_question = []
    data_sentence_choice = []
    for word in (takken.loc[i, 'Question']).split():
        data_sentence_question.append(word)
    data_question.append(data_sentence_question)
    for word in (takken.loc[i, 'Choice']).split():
        data_sentence_choice.append(word)
    data_choice.append(data_sentence_choice)

##############################################################
#vocabに保存
"""
with open('question.vocab','wb') as f:
    pickle.dump(question_words, f)
with open('choice.vocab', 'wb') as g:
    pickle.dump(choice_words, g)

datasize = takken.shape[0]
"""
######################################################################
MIN_COUNT = 1
# 特殊なトークンを事前に定義します
PAD_TOKEN = '<PAD>'
BOS_TOKEN = '<S>'
EOS_TOKEN = '</S>'
UNK_TOKEN = '<UNK>'
PAD = 0
BOS = 1
EOS = 2
UNK = 3
word2id = {
    PAD_TOKEN: PAD,
    BOS_TOKEN: BOS,
    EOS_TOKEN: EOS,
    UNK_TOKEN: UNK,
    }

def sentence_to_ids(vocab, sentence):
    ids = [vocab.word2id.get(word, UNK) for word in sentence]
    ids = [BOS] + ids + [EOS]
    return ids

def pad_seq(sen, max_length):
    if len(sen) <= max_length:
        sen += [PAD] * (max_length - len(sen))
    else:
        sen = sen[:max_length]
    return sen
print(data_choice[0])
vocab = Vocab(word2id=word2id)
vocab.build_vocab(data_choice, min_count=MIN_COUNT)
print("語彙数\t:", len(vocab.word2id))


deviceはGPUを使用します
data size is 9442
['<BOS>', '建設', '業法', 'による', '建設', '業', 'の', '許可', 'を', '受け', 'て', 'いる', 'A', 'が', '建築', '請負', '契約', 'に', '付帯', 'し', 'て', '取り決め', 'た', '約束', 'を', '履行', 'する', 'ため', '建築', 'し', 'た', '共同', '住宅', 'の', '売買', 'の', 'あっせん', 'を', '反復', '継続', 'し', 'て', '行う', '場合', '<EOS>']
語彙数	: 4523


In [ ]:
def generate_samples(model, batch_size, generated_num, output_file):
    samples= []
    for _ in range(int(generated_num / batch_size)):
        sample = model.sample().cpu().data.numpy().tolist()
        samples.extend(sample)

    with open(output_file, 'w') as fout:
        for sample in samples:
            string = ' '.join([str(s) for s in sample]) + "\n"
            fout.write(string)
            
def compute_loss(model, data_loader, criterion, optimizer=None, is_train=True):
    model.train(is_train)
    total_loss = 0.
    total_batches = 0.
    
    for batch_X, batch_Y in data_loader:
        pred_Y = model(batch_X)
        loss = criterion(pred_Y, batch_Y.contiguous().view(-1))
        total_loss += loss.item()
        total_batches += 1
        
        if is_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    data_loader.reset()
    loss = total_loss / total_batches
    return loss

In [ ]:
# Generator Hyper-parameters
G_EMBEDDING_SIZE = 32 # 埋め込みベクトルの次元数
G_HIDDEN_SIZE = 32 # LSTMの隠れ状態ベクトルの次元数
G_MAX_LENGTH = 80 # 系列の長さ
G_PRE_NUM_EPOCHS = 120 # 事前学習を行うエポック数
#G_PRE_NUM_EPOCHS = 1
G_BATCH_SIZE = 64 # バッチサイズ

# Discriminator Hyper-parameters
D_EMBEDDING_SIZE = 64
D_FILTER_SIZES = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20] # CNNに使うフィルターのサイズ
D_NUM_FILTERS = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160] # CNNに使うフィルターの数
D_DROPOUT_PROB = 0.75 # Dropoutの確率
D_PRE_NUM_EPOCHS = 50
#D_PRE_NUM_EPOCHS = 1
D_BATCH_SIZE = 64

# Basic Training Parameters
ADV_N_BATCHES = 200 # 敵対的学習を行うエポック数
#ADV_N_BATCHES = 1
POSITIVE_FILE = './save/real_data.txt'
NEGATIVE_FILE = './save/generator_sample.txt'
GENERATED_NUM = 10000
#GENERATED_NUM = 1000 # 敵対的学習のためにGeneratorに出力させるサンプル数

# 保存用ディレクトリがなければ作成する
if not os.path.exists('./save'):
    os.mkdir('./save')

In [ ]:
id_text = []
for sen in data_choice:
    id_sen =sentence_to_ids(vocab, sen)
    id_sen = pad_seq(id_sen, G_MAX_LENGTH)
    id_text.append(id_sen)
    
with open(POSITIVE_FILE, 'w') as f:
    for id_sen in id_text:
        f.write(' '.join([str(w) for w in id_sen])+ '\n')

In [ ]:
vocab_size = len(vocab.word2id)
generator = Generator(vocab_size, G_BATCH_SIZE, G_EMBEDDING_SIZE, G_HIDDEN_SIZE, G_MAX_LENGTH).to(device)
discriminator = Discriminator(vocab_size, D_BATCH_SIZE, D_EMBEDDING_SIZE,
                D_NUM_FILTERS, D_FILTER_SIZES, dropout_prob=D_DROPOUT_PROB).to(device)

gen_data_loader = GenDataLoader(POSITIVE_FILE, G_BATCH_SIZE)
gen_criterion = nn.NLLLoss()
gen_optimizer = optim.Adam(generator.parameters())
for epoch_id in range(1, G_PRE_NUM_EPOCHS + 1):
    gen_data_loader.reset()
    train_loss = compute_loss(
        generator, gen_data_loader, gen_criterion, optimizer=gen_optimizer, is_train=True)
    print("Epoch:{}, GenLoss:{:.4f}".format(epoch_id, train_loss))

Epoch:1, GenLoss:5.6965
Epoch:2, GenLoss:3.6070
Epoch:3, GenLoss:3.3424
Epoch:4, GenLoss:3.1393
Epoch:5, GenLoss:2.9169
Epoch:6, GenLoss:2.7238
Epoch:7, GenLoss:2.5752
Epoch:8, GenLoss:2.4650
Epoch:9, GenLoss:2.3790
Epoch:10, GenLoss:2.3088
Epoch:11, GenLoss:2.2498
Epoch:12, GenLoss:2.1991
Epoch:13, GenLoss:2.1551
Epoch:14, GenLoss:2.1163
Epoch:15, GenLoss:2.0814
Epoch:16, GenLoss:2.0497
Epoch:17, GenLoss:2.0206
Epoch:18, GenLoss:1.9937
Epoch:19, GenLoss:1.9685
Epoch:20, GenLoss:1.9452
Epoch:21, GenLoss:1.9233
Epoch:22, GenLoss:1.9029
Epoch:23, GenLoss:1.8837
Epoch:24, GenLoss:1.8658
Epoch:25, GenLoss:1.8489
Epoch:26, GenLoss:1.8330
Epoch:27, GenLoss:1.8179
Epoch:28, GenLoss:1.8034
Epoch:29, GenLoss:1.7895
Epoch:30, GenLoss:1.7763
Epoch:31, GenLoss:1.7636
Epoch:32, GenLoss:1.7514
Epoch:33, GenLoss:1.7397
Epoch:34, GenLoss:1.7285
Epoch:35, GenLoss:1.7176
Epoch:36, GenLoss:1.7070
Epoch:37, GenLoss:1.6969
Epoch:38, GenLoss:1.6872
Epoch:39, GenLoss:1.6777
Epoch:40, GenLoss:1.6687
Epoch:41,

In [ ]:
dis_criterion = nn.NLLLoss()
dis_optimizer = optim.Adam(discriminator.parameters())
for epoch_id in range(1, D_PRE_NUM_EPOCHS + 1):
    generate_samples(generator, D_BATCH_SIZE, GENERATED_NUM, NEGATIVE_FILE)
    dis_data_loader = DisDataLoader(POSITIVE_FILE, NEGATIVE_FILE, D_BATCH_SIZE)
    for _ in range(3):
        loss = compute_loss(
            discriminator, dis_data_loader, dis_criterion, optimizer=dis_optimizer, is_train=True)
        print("Epoch:{}, DisLoss: {:.4f}".format(epoch_id, loss))
        break

Epoch:1, DisLoss: 0.1438
Epoch:2, DisLoss: 0.0005
Epoch:3, DisLoss: 0.0001
Epoch:4, DisLoss: 0.0000
Epoch:5, DisLoss: 0.0086
Epoch:6, DisLoss: 0.0010
Epoch:7, DisLoss: 0.0000
Epoch:8, DisLoss: 0.0000
Epoch:9, DisLoss: 0.0000
Epoch:10, DisLoss: 0.0000
Epoch:11, DisLoss: 0.0000
Epoch:12, DisLoss: 0.0000
Epoch:13, DisLoss: 0.0000
Epoch:14, DisLoss: 0.0000
Epoch:15, DisLoss: 0.0000
Epoch:16, DisLoss: 0.0000
Epoch:17, DisLoss: 0.0000
Epoch:18, DisLoss: 0.0000
Epoch:19, DisLoss: 0.0217
Epoch:20, DisLoss: 0.0009
Epoch:21, DisLoss: 0.0002
Epoch:22, DisLoss: 0.0007
Epoch:23, DisLoss: 0.0022
Epoch:24, DisLoss: 0.0000
Epoch:25, DisLoss: 0.0005
Epoch:26, DisLoss: 0.0000
Epoch:27, DisLoss: 0.0091
Epoch:28, DisLoss: 0.0147
Epoch:29, DisLoss: 0.0034
Epoch:30, DisLoss: 0.0000
Epoch:31, DisLoss: 0.0169
Epoch:32, DisLoss: 0.0007
Epoch:33, DisLoss: 0.0014
Epoch:34, DisLoss: 0.0000
Epoch:35, DisLoss: 0.0000
Epoch:36, DisLoss: 0.0000
Epoch:37, DisLoss: 0.0000
Epoch:38, DisLoss: 0.0000
Epoch:39, DisLoss: 0.

# adversarial Training

In [ ]:
rollout = Rollout(generator, 0.8)

gen_gan_criterion = nn.NLLLoss(reduce=False)
gen_gan_optimizer = optim.Adam(generator.parameters())
dis_criterion = nn.NLLLoss()
dis_optimizer = optim.Adam(discriminator.parameters())
for batch_id in range(1, ADV_N_BATCHES + 1):
    for it in range(1):
        samples = generator.sample()
        start_tokens = torch.empty(G_BATCH_SIZE, 1).fill_(BOS).type(torch.long)
        start_tokens = start_tokens.to(device)
        inputs = torch.cat([start_tokens, samples], dim= 1)[:, :-1].contiguous()
        targets = samples.contiguous().view((-1,))
        
        rewards = rollout.get_reward(samples, 16, discriminator)
        rewards = torch.Tensor(rewards).contiguous().view((-1)).to(device)
        log_prob = generator.forward(inputs)
        train_loss = gen_gan_criterion(log_prob, targets)
        train_loss = (train_loss*rewards).mean()
        
        gen_gan_optimizer.zero_grad()
        train_loss.backward()
        gen_gan_optimizer.step()
        
    print('Batch: {}, TrainLoss:{:.4f}'.format(batch_id, train_loss.item()))
    
    rollout.update_params()
    
    for _ in range(4):
        generate_samples(generator, G_BATCH_SIZE, GENERATED_NUM, NEGATIVE_FILE)
        dis_data_loader = DisDataLoader(POSITIVE_FILE, NEGATIVE_FILE, D_BATCH_SIZE)
        for _ in range(2):
            loss = compute_loss(discriminator, dis_data_loader,dis_criterion,
                               optimizer=dis_optimizer, is_train=True)

/home/develop/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Batch: 1, TrainLoss:0.0003
Batch: 2, TrainLoss:0.0018
Batch: 3, TrainLoss:0.0032
Batch: 4, TrainLoss:0.0011
Batch: 5, TrainLoss:0.0000
Batch: 6, TrainLoss:0.0000
Batch: 7, TrainLoss:0.0000
Batch: 8, TrainLoss:0.0029
Batch: 9, TrainLoss:0.0000
Batch: 10, TrainLoss:0.0003
Batch: 11, TrainLoss:0.0010
Batch: 12, TrainLoss:0.0000
Batch: 13, TrainLoss:0.0000
Batch: 14, TrainLoss:0.0000
Batch: 15, TrainLoss:0.0007
Batch: 16, TrainLoss:0.0000


In [ ]:
class TextGenerator(nn.Module):
    def __init__(self, num_emb, batch_size, emb_dim, hidden_dim,
                 sequence_length):
        """
        :param num_emb: int, 語彙の総数
        :param batch_size: int, ミニバッチのサイズ
        :param emb_dim: int, 埋め込みベクトルの次元数
        :param hidden_dim: int, 隠れ状態ベクトルの次元数
        :param sequence_length: int, 入出力系列の長さ
        """
        super(TextGenerator, self).__init__()
        self.num_emb = num_emb
        self.batch_size = batch_size
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.sequence_length = sequence_length
        
        # 埋め込み層
        self.embedding = nn.Embedding(self.num_emb, self.emb_dim)
        # LSTM
        self.lstm = nn.LSTM(self.emb_dim, self.hidden_dim, 1, batch_first=True)
        # 全結合層
        self.linear = nn.Linear(self.hidden_dim, self.num_emb)

    def forward(self, inputs):
        """
        ターゲット系列を全時刻での入力として出力を計算
        :param inputs: torch.Tensor, (batch_size, sequence_length)
        :return outputs: torch.Tensor, (batch_size*sequence_length, num_emb)
        """
        N = inputs.size(0) # batch_size
        embed = self.embedding(inputs) # (batch_size, sequence_length, emb_dim)
        h0, c0 = self.init_hidden(N) # 隠れ状態ベクトルの初期化
        h = (h0, c0)
        self.lstm.flatten_parameters()
        hidden, h = self.lstm(embed, h) # hidden:(batch_size, sequence_length, hidden_dim)
        lin = self.linear(hidden) # (batch_size, sequence_length, num_emb)
        outputs = F.log_softmax(lin, dim=-1) # (batch_size, sequence_length, num_emb)
        outputs = outputs.view(-1, self.num_emb) # (batch_size * sequence_length, num_emb)
        return outputs

    def step(self, x, h, c):
        """
        時刻をtからt+1に1つだけ進めます
        :param x: torch.Tensor, 時刻tの出力かつ時刻t+1の入力
        :param h, c: torch.Tensor, 時刻tの隠れ状態ベクトル
        :return pred: torch.Tensor, 時刻t+1の出力
        :return h, c: torch.Tensor, 時刻t+1の隠れ状態ベクトル
        """
        embed = self.embedding(x) # embed:(batch_size, 1, emb_dim)
        self.lstm.flatten_parameters()
        y, (h, c) = self.lstm(embed, (h, c)) # y:(batch_size, 1, hidden_dim)
        pred = F.softmax(self.linear(y), dim=-1) # (batch_size, 1, num_emb)
        return pred, h, c

    def sample(self, x=None):
        """
        Generaterでサンプリングするメソッド
        :param x: None or torch.Tensor
        :param output: torch.Tensor, (batch_size, sequence_length)
        """
        flag = False # 時刻0から始める(True)か否か(False)
        if x is None:
            flag = True
        if flag:
            x = torch.empty(self.batch_size, 1).fill_(1).long().to(device) # BOS == 1
        h, c = self.init_hidden(self.batch_size)

        samples = []
        if flag:
            for i in range(self.sequence_length):
                output, h, c = self.step(x, h, c) # output:(batch_size, 1, num_emb)
                output = output.squeeze(1) # (batch_size, num_emb)
                x = output.multinomial(1) # (batch_size, 1), 次の時刻の入力を多項分布からサンプリング
                samples.append(x)
        else:
            given_len = x.size(1)
            lis = x.chunk(x.size(1), dim=1) # sequence_length方向に分割
            for i in range(given_len):
                output, h, c = self.step(lis[i], h, c)
                samples.append(lis[i])
            output = output.squeeze(1)
            x = output.multinomial(1)
            for i in range(given_len, self.sequence_length):
                samples.append(x)
                output, h, c = self.step(x, h, c)
                output = output.squeeze(1)
                x = output.multinomial(1)
        output = torch.cat(samples, dim=1)
        return output

    def init_hidden(self, N):
        """
        LSTMの隠れ状態ベクトルを初期化します。
        :param N: int, ミニバッチのサイズ
        """
        h0 = torch.zeros(1, N, self.hidden_dim).to(device)
        c0 = torch.zeros(1, N, self.hidden_dim).to(device)
        return h0, c0


In [ ]:
def ids_to_sentence(vocab, ids):
    # IDのリストを単語のリストに変換する
    return [vocab.id2word[int(_id)] for _id in ids]


for line in open(NEGATIVE_FILE).readlines():
    ids_line = []
    for word in line[:-1].split(' '):
        ids_line.append(word)
    print(ids_to_sentence(vocab, ids_line))